### Multi GPU 활용을 통한 학습 속도 향상  

#### 목적 

아래에 리스트업 되는 GPU를 어떻게 하면 학습에서 잘 활용할 것인가? 

In [1]:
!nvidia-smi

Thu Jan 25 15:42:20 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 387.26                 Driver Version: 387.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:01:00.0  On |                  N/A |
|  0%   31C    P8     8W / 250W |     50MiB /  8110MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   31C    P8     8W / 250W |      2MiB /  8114MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### parallel training 

- 현시점 대부분의 딥러닝 프레임웍은 알고리즘 분산보다는 데이터 분산으로 병렬학습을 지원하고 있다. 
- 병렬 학습에 사용되는 모든 GPU는 동일한 모형 아키텍처의 복사본을 각기 GPU에 보유하고 있다. 
  - 따라서 하나의 GPU에 모형 아케텍처가 모두 올라가지 않으면 학습을 할 수 없다. 
  - GPU 여러장을 하나의 GPU처럼 활용하는 기법은 현재 컨셉으로만 존재한다. 
  
### data parallel training 

- pseudo-code

```
def train_batch(data, k):
    split data into k parts
    for i = 1, ..., k:  # run in parallel
        compute grad_i w.r.t. weight_i using data_i on the i-th GPU
    grad = grad_1 + ... + grad_k
    for i = 1, ..., k:  # run in parallel
        copy grad to i-th GPU
        update weight_i by using grad
```

### Gluon parallel training

- in training code
```
GPU_COUNT = 2
ctx= [mx.gpu(i) for i in range(GPU_COUNT)]
....
        for i, (x_data, y_data, z_data) in enumerate(tr_data_iterator):
            x_data_l = gluon.utils.split_and_load(x_data, ctx)
            y_data_l = gluon.utils.split_and_load(y_data, ctx)
            z_data_l = gluon.utils.split_and_load(z_data, ctx)
            with autograd.record():
                losses = [loss(model(x, y), z) for x, y, z in zip(x_data_l, y_data_l, z_data_l)]
            for l in losses:
                l.backward()
            trainer.step(x_data.shape[0])
....
```


### 멀티 GPU의 장/단점 

장점 

- 한번에 많은 배치를 학습할 수 있다. 

단점 

- 배치를 늘리지 않는다면 효과가 거의 없거나 오히려 학습 속도를 저해한다. 
- 배치라는 하이퍼파라메터가 바뀌기 때문에 싱글GPU에서 학습한 모형과 다른 모형이 도출된다.


### 8장의 GPU 중에서 마지막 4장의 GPU만 쓰고 싶다면.. 

아래의 코드를 Python 코드의 앞부분에서 실행해준다. 

```
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"
```


### 테스트

#### 환경 

- 한영기계번역모형(seq2seq-atten)
- A : GTX 1080 GPU 8Gb x 2
- B : DGX Tesla V100 16Gb x 4
    
    
- A환경에서는 1배치에 60개의 문장 학습 가능(메모리 한계)
- B환경을 사용하면서 1배치에 240개 문장 학습 가능(GPU 메모리 2배 x GPU 숫자 2배)



#### 1배치 학습 시간 

- A

![image.png](a.png)


- B
![image.png](b.png)


약 5배 학습 속도가 향상됨. 
- 산술보다 빠른건 DGX에서 지원하는 [NVlink](https://devblogs.nvidia.com/dgx-1-fastest-deep-learning-system/) 기술 때문인것으로 추정됨
- MXNet은 GPU가 늘어날 수록 선형적으로 학습 성능 향상이 가능하다는걸 알 수 있음 